## SUPPLY CHAIN NETWORK DESIGN TO SUPPORT BIOFUEL PRODUCTION
<p> A company has decided to produce bioethanol in the state of Texas. The company needs to design a supply chain consisting of suppliers, hubs and biorefineries for the conversion of raw material (i.e., biomass) into biofuel. </p>
<p> The potential locations to open hubs correspond to train stations because the transportation mode utilized to move the raw material from the hubs to the biorefineries is train, while truck is the transportation mode utilized to move the biomass from the counties to the hubs.</p>
<p> This project is to minimize the investment and transportation costs by finding the optimal number of hubs and biorefineries that the company needs to install as well as the flows between suppliers-hubs and hubs-biorefineries.</p>


<p> PuLP is the Python library installed, imported and used in this project for linear optimization </p>

In [1]:
# Import the necessary libraries
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import pulp
from pulp import*

## Data Importation and Formatting

In [2]:
suppliers = pd.read_csv('TX_suppliers.csv')
suppliers['supply'] = round(suppliers.supply, 2)
suppliers = suppliers.drop(['index'], axis =1)
supplies = suppliers.supply
total_supply = sum(suppliers.supply)

hubs = pd.read_csv('TX_hubs.csv')
hubs = hubs.drop(['index'], axis =1)
Hub_Capacity = hubs.capacity
Hub_Invest = hubs.invest.iloc[0]
Hubs = hubs['hub']

road = pd.read_csv('TX_roads.csv')
road['cost'] = round(road.cost, 2)
road = road.drop(['index'], axis =1)

plants = pd.read_csv('TX_plants.csv')
Yield = plants['yield'].iloc[0]
plants['plt_capacity'] = plants.capacity/plants['yield']
plants = plants.drop(['index'], axis =1)
Plt_Invest = plants.invest.iloc[0]

rail = pd.read_csv('TX_railroads.csv')
rail['cost'] = round(rail.cost, 2)
rail['plt_invest'] = len(rail)*[plants.invest.iloc[0]]
rail = rail.drop(['index'], axis =1)
loading = rail.loading

network = pd.read_csv('TX_network.csv')
Demand = network.demand.iloc[0]
demand = Demand/Yield

unmet_Demand = demand - total_supply

## Introducing 3rd Party Supplier
<p> Since the sum of the county supplies can not meet the demand, a 3rd party supplier is introduced with an estimated unit cost </p>

In [3]:
suppliers.loc[254] = ['3_party', unmet_Demand]
suppliers['supply'] = round(suppliers.supply, 2)

In [4]:
# 3rd Party Estimate Cost
Third_cost = sum(road.cost)/len(road)
Third_cost = round(Third_cost)
Third_amount = "${:,.2f}".format(Third_cost)
print('The 3rd-Party estimate cost is ' + Third_amount)

The 3rd-Party estimate cost is $92.00


In [5]:
# Adding 3rd Party Supplier to road table
n = 1303
County = ['3_party'] * n
Distance = [0] * n
Cost = [Third_cost] * n
Third = {'county': County, 'distance':Distance, 'cost':Cost}
Third = pd.DataFrame(Third)
Third['hubs'] = Hubs

In [6]:
road = road.append(Third, ignore_index = True) 

,cost,county,distance,hubs
0,197.28,48001,1218.9736,131
1,53.03,48001,299.0238,199
2,147.18,48001,899.4465,229
3,53.21,48001,300.1810,512
4,53.04,48001,299.0745,560
5,192.24,48001,1186.8121,635
6,192.22,48001,1186.6888,636
7,178.89,48001,1101.6754,637
8,115.41,48001,696.8238,638
9,118.62,48001,717.3399,639


In [7]:
# Setting the indices
hubs = hubs.set_index(['hub'])
suppliers = suppliers.set_index(['county'])
plants = plants.set_index(['plant'])
road = road.set_index(['county', 'hubs'])
rail = rail.set_index(['hubs','plant'])

## Data Exploration and Optimization

In [8]:
# Decision Variables Defintion
road_supply = pulp.LpVariable.dicts('road_supply', [(i, j) for i in suppliers.index
                                                    for j in hubs.index], lowBound=0, cat='Continuous')

hub_status = pulp.LpVariable.dicts("hub_status", [j for j in hubs.index], cat='Binary')

rail_supply = pulp.LpVariable.dicts("rail_supply",[(j, k) for j in hubs.index
                                                   for k in plants.index], lowBound=0, cat='Continuous')

plt_status = pulp.LpVariable.dicts("plt_status",[k for k in plants.index], cat='Binary')

## Model

In [9]:
# Model Initialization
model = pulp.LpProblem("cost minimising supply network", pulp.LpMinimize)

In [10]:
# Objective Function
model += pulp.lpSum(
    [[road_supply[i, j] * road.loc[(i, j), 'cost']]  + 
     [hub_status[j] * hubs.loc[j, 'invest']] for i in suppliers.index for j in hubs.index] + [[rail_supply[j, k] * rail.loc[(j, k), 'cost']] + 
     [plt_status[k] * plants.loc[k, 'invest']] for j in hubs.index for k in plants.index]) 

In [11]:
# Demand Constraints
model += pulp.lpSum([road_supply[i, j] for i in suppliers.index for j in hubs.index]) == demand
model += pulp.lpSum([rail_supply[j, k] for j in hubs.index for k in plants.index]) == demand

In [12]:
# Hub Capacity Constraint
for j in hubs.index:
    model += pulp.lpSum([road_supply[i, j] for i in suppliers.index]) <= hubs.loc[j, 'capacity'] * hub_status[j]

In [13]:
# Plant Capacity Constraint
for k in plants.index:
    model += pulp.lpSum([rail_supply[j, k] for j in hubs.index]) <= plants.loc[k, 'plt_capacity'] * plt_status[k]

In [14]:
# Model Status
model.solve()
pulp.LpStatus[model.status]

'Optimal'

## Result

In [15]:
cost = pulp.value(model.objective)
amount = "${:,.2f}".format(cost)
print('The optimal cost is ' + amount)

The optimal cost is $1,725,868,653,500.00


In [16]:
# Hub Supply Table Formulation
rd_output = []
for i, j in road_supply:
    var_output = {
        'county': i,
        'hubs': j,
        'road_supply': road_supply[(i, j)].varValue,
        'hub_status': hub_status[j].varValue
    }
    rd_output.append(var_output)
rd_output_df = pd.DataFrame.from_records(rd_output).sort_values(['county', 'hubs'])
rd_output_df.set_index(['county', 'hubs'], inplace=True)

## Hub Supply Output

In [17]:
rd = []
for i, j in road_supply:
    if rd_output_df.hub_status[i, j] == 1 and rd_output_df.road_supply[i, j] > 0:
        output = {
            'hubs': j,
            'road_supply': rd_output_df.road_supply[i, j],
            'hub_status': rd_output_df.hub_status[i, j] 
        }
        rd.append(output)
rd = pd.DataFrame.from_records(rd).sort_values('hubs')
rd.set_index('hubs', inplace=True)
rd

,hub_status,road_supply
hubs,,
512,1.0,300000.000
17246,1.0,300000.000
17318,1.0,300000.000
17387,1.0,300000.000
17399,1.0,300000.000
17482,1.0,300000.000
17517,1.0,300000.000
17623,1.0,300000.000
17695,1.0,63407.767


In [18]:
# Hub Supply Document Exportation
rd.to_excel('Optimal Hubs.xlsx')

In [19]:
# Plant Supply Table Formulation
rl_output = []
for j, k in rail_supply: 
    var_output = {
        'hubs': j,
        'plant': k,
        'rail_supply': rail_supply[(j, k)].varValue,
        'plt_status': plt_status[k].varValue
    }
    rl_output.append(var_output)
rl_output_df = pd.DataFrame.from_records(rl_output).sort_values(['hubs', 'plant'])
rl_output_df.set_index(['hubs', 'plant'], inplace=True)

## Plant Supply Ouput

In [20]:
rl = []
for j, k in rail_supply:
    if rl_output_df.plt_status[j, k] == 1 and rl_output_df.rail_supply[j, k] > 0:
        output = {
            'plants': k,
            'rail_supply': rl_output_df.rail_supply[j, k],
            'plt_status': rl_output_df.plt_status[j, k] 
        }
        rl.append(output)
rl = pd.DataFrame.from_records(rl).sort_values('plants')
rl.set_index('plants', inplace=True)
rl

,plt_status,rail_supply
plants,,
543,1.0,655447.00
9088,1.0,655447.00
9091,1.0,655447.00
9104,1.0,655447.00
9142,1.0,655447.00
9167,1.0,464384.73
9188,1.0,655447.00
9203,1.0,655447.00
10060,1.0,655447.00


In [21]:
# Plant Supply Document Exportation
rl.to_excel('Optimal Plants.xlsx')